# NLP Pionner — Twitter Hate Speech Detection

## Modeling Families & Business-Aligned Explainable Models

**Goal:** Explore one model per family (Linear, Ensemble, Boosting) with business constraint — transparency & interpretability.

**Families:**
- Linear: Logistic Regression (L1 penalty) — base model, interpretable.
- Ensemble: Bagging (Decision Tree) — moderate transparency via permutation importance.
- Boosting: Explainable Boosting Machine (EBM) — glass-box additive model.
- *(Optional)* Stacking if time allows.


In [ ]:
!pip install -q kaggle emoji regex scikit-learn interpret matplotlib
from google.colab import files
print('Upload kaggle.json (Kaggle > Account > Create API Token)')
files.upload()
import os, pandas as pd, numpy as np, re, regex, emoji
!mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vkrahul/twitter-hate-speech -p ./data --unzip
df = pd.read_csv('./data/train_E6oV3lV.csv')
df.columns = [c.lower() for c in df.columns]
if 'tweet' in df.columns: df = df.rename(columns={'tweet':'text'})
df['text'] = df['text'].fillna('')
df.head()

In [ ]:
# Cleaning
URL_RE=re.compile(r'https?://\S+|www\.\S+'); MENTION=re.compile(r'@\w+'); HASHTAG=re.compile(r'#\w+'); NUM=re.compile(r'\d+'); SPACE=re.compile(r'\s+')
def strip_emoji(s): import emoji as em; return em.replace_emoji(s, replace='')
def normalize_hashtags(s): return HASHTAG.sub(lambda m: m.group(0)[1:], s)
def clean_text(s:str)->str:
    s=s.lower(); s=URL_RE.sub(' ',s); s=MENTION.sub(' @user ',s); s=normalize_hashtags(s)
    s=NUM.sub(' <num> ',s); s=strip_emoji(s); s=regex.sub(r'[^\p{L}\p{N}\s\.,!\?\']+',' ',s); s=SPACE.sub(' ',s).strip(); return s
df['text_clean']=df['text'].astype(str).map(clean_text)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test=train_test_split(df['text_clean'],df['label'],test_size=0.2,stratify=df['label'],random_state=42)
tfidf_word=TfidfVectorizer(ngram_range=(1,2),max_features=30000,min_df=2)
Xtr=tfidf_word.fit_transform(X_train); Xte=tfidf_word.transform(X_test)

In [ ]:
# 1️⃣ Linear Model: Logistic Regression (L1)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
logreg=LogisticRegression(penalty='l1',solver='liblinear',class_weight='balanced',max_iter=400)
logreg.fit(Xtr,y_train)
pred_lr=logreg.predict(Xte)
print('Logistic Regression (L1) Results:\n')
print(classification_report(y_test,pred_lr,digits=3))

import numpy as np
feature_names=np.array(tfidf_word.get_feature_names_out())
coefs=logreg.coef_[0]
top_pos_idx=np.argsort(coefs)[-20:][::-1]; top_neg_idx=np.argsort(coefs)[:20]
print('\nTop Positive Features (Hate):')
for f,c in zip(feature_names[top_pos_idx],coefs[top_pos_idx]): print(f'{f:25s} {c: .3f}')
print('\nTop Negative Features (Non-Hate):')
for f,c in zip(feature_names[top_neg_idx],coefs[top_neg_idx]): print(f'{f:25s} {c: .3f}')

In [ ]:
# 2️⃣ Ensemble Model: Bagging (Decision Trees)
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag=BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=6,min_samples_leaf=5,class_weight='balanced'),n_estimators=50,random_state=42,n_jobs=-1)
bag.fit(Xtr,y_train)
pred_bag=bag.predict(Xte)
print('Bagging Trees Results:\n')
print(classification_report(y_test,pred_bag,digits=3))

# Permutation importance on subset
from sklearn.inspection import permutation_importance
idx_small=np.random.RandomState(42).choice(Xte.shape[1],size=800,replace=False)
Xte_small=Xte[:,idx_small]
feat_small=np.array(tfidf_word.get_feature_names_out())[idx_small]
perm=permutation_importance(bag,Xte_small.toarray(),y_test,n_repeats=5,random_state=42,n_jobs=-1)
order=np.argsort(perm.importances_mean)[::-1][:20]
print('\nTop 20 features by permutation importance (Bagging):')
for f,imp in zip(feat_small[order],perm.importances_mean[order]): print(f'{f:25s} {imp: .5f}')

In [ ]:
# 3️⃣ Boosting Model: Explainable Boosting Machine (EBM)
from interpret.glassbox import ExplainableBoostingClassifier
ebm=ExplainableBoostingClassifier(interactions=5,outer_bags=8,inner_bags=0,max_bins=256,learning_rate=0.02,validation_size=0.15,random_state=42)
Xtr_d=Xtr.tocsc()[:,:20000].astype('float32').toarray()
Xte_d=Xte.tocsc()[:,:20000].astype('float32').toarray()
ebm.fit(Xtr_d,y_train)
pred_ebm=ebm.predict(Xte_d)
print('EBM Results:\n')
print(classification_report(y_test,pred_ebm,digits=3))
ebm_global=ebm.explain_global()
print('Top features (EBM):', ebm_global.data()['names'][:15])

In [ ]:
# 4️⃣ Optional: Stacking (only if allowed by business)
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
svm=CalibratedClassifierCV(LinearSVC(class_weight='balanced'),cv=5)
stack=StackingClassifier(estimators=[('lr',logreg),('bag',bag),('svm',svm)],final_estimator=LogisticRegression(max_iter=300,class_weight='balanced'),n_jobs=-1)
stack.fit(Xtr,y_train)
pred_stack=stack.predict(Xte)
print('Stacking Results:\n')
print(classification_report(y_test,pred_stack,digits=3))

## ✅ Model Selection Logic (Business-Aligned)
- Prioritize macro-F1 and recall on the hate class.
- If EBM performs within 1% of Logistic Regression F1 → **choose EBM** (better nonlinearity + explainability).
- Otherwise choose **Logistic Regression (L1)** for full transparency.
- Bagging = challenger; use permutation importance and PDPs for interpretability.
